In [2]:
import cohere
import time
import pandas as pd
import matplotlib.pyplot as plt

# Set Cohere API key
cohere_api_key = '47amNsxWIPexaTSxCmteAskkxXJ9XVfB2nClzP0j' 
co = cohere.Client(cohere_api_key)

# Function to perform sentiment analysis
def get_sentiment(review):
    prompt = f"""
    Analyze the sentiment of the following review:
    "{review}"
    Assign one of the following labels: "positive", "negative", "neutral".
    Make sure to answer only with the label.
    """
    response = co.generate(
        model='command-xlarge-nightly',
        prompt=prompt,
        max_tokens=10,
        temperature=0.3
    )
    return response.generations[0].text.strip()

# Function to extract the most important phrase influencing the sentiment
def get_extractive_self_explanation(review):
    prompt = f"""
    You have just analyzed the following review:
    "{review}"
    What is the most important phrase influencing your sentiment assessment? Provide only the phrase as a string.
    """
    response = co.generate(
        model='command-xlarge-nightly',
        prompt=prompt,
        max_tokens=20,
        temperature=0.3
    )
    return response.generations[0].text.strip()

# Function to generate a counterfactual version of the review
def get_counterfactual_review(review):
    prompt = f"""
    You have analyzed the following review:
    "{review}"
    Now, provide a version of the review that would flip your sentiment assessment while changing as few words as possible. Make sure to answer with only the new version.
    """
    response = co.generate(
        model='command-xlarge-nightly',
        prompt=prompt,
        max_tokens=150,
        temperature=0.3
    )
    return response.generations[0].text.strip()

# Example usage with the complete reviews
reviews = [
    "آش دوغ ترش نبود و آش شله قلمکار اصلا کیفیت مناسبی نداشت. پر از آرد و حبوبات نپخته خیلی بد مزه بود و خیلی شیرین بود",
    "مثل همیشه خوب و باکیفیت. هر وقت هم که داخل قسمت توضیحات چیزی خواستیم انجام دادن، در حالی که خیلی جاها اصلا توجه به توضیحات نمیکنن.",
    "غذا خوب بود ولی قیمت غذا و قیمت ارسال کمی زیاده.",
    "جوجه کباب خیلی خوشمزه و ترد بود و اصلا خشک یا سوخته نبود، برنج هم به خوبی دم کشیده بود و تهدیگ مناسبی داشت فقط حجم برنج می‌توانست بیشتر باشد، هزینه حمل ٥ تومنی از دولت تا پل صدر کمی زیاد است لطفا در این مورد تجدید نظر نمایید، بسته بندی هم خوب بود غذا گرم رسید",
    "غذای جوجه برنجش شفته شده بود",
    "نان خامه‌ای خوب تخمه خیلی تازه نبود",
    "چرا به کل سفارش مالیات بر ارزش افزوده اضافه میشه",
    "کیفیت نسبتا خوب ولی حجم مرغ سوخاری ۲ تکه آن ۲ تکه نبود و عملا یک تکه و نیم بود یک تکه سینه و یک تکه کتف",
    "سرعت و کیفیت ارسال بالا",
    "لطفا موجود کن دیجی",
    "من این لپتاپو برای کارای روزمره ی برادرم گرفتم و ویندوز 10 روز نصب کردم تا الان خیلی عالی جواب میده و خیلی خوبه اگه برای کارای روزمره میخواین من واقعا این لپتاپو بهتون پیشنهاد میکنم  اولش به خاطر امتیاز پایینی که دوستان داده بودن مردد بودم  ولی بالاخره گرفتم و فهمیدم که امتیازش واقعا بالا تر از این حرفا باید باشه",
    "این لب تاپ واقعا نسبت به قیمتش به صرفس حتما پیشنهاد میکنم به کسایی که با قیمت کم ویژگی های بالایی میخوان حتما بهش فکرکنن",
    "من امروز به دستم رسید  یه نکته که هست اینه که این لپ تاپ 2 تا کارت گرافیک داره یه اینتل 5500 و یه amd 2g اینو من تو دیجیکالا ندیدم  نسبت به عکس خیلی زیبا تر و شیک تره  کیفیت ساختش خوبه  صداش برای یه لپ تاپ خوبه زیاد نیست ولی قابل قبوله اگه پورت vga احتیاج ندارین واقعا یه چیز اضافیه نباشه بهتره  تو این رنج قیمت مناسبه  داغ نمیکنه البته من هنوز باهاش کاره خاصی انجام ندادم  ال سی دی شفاف مشکلات خودشو داره ولی واسه این لپ تاپ خوبه  اگه یه لپ تاپه زیبا و ساده میخواید که کارای معمولی باهاش انجام بدین شک نکنین پشیمون نمیشین ولی برا بازی پیشنهاد نمیکنم ممنون بابت ارسال سر وقت از دیجی کالا",
    "بسیاردستگاه عالی و زیبا و کار آمدی هست",
    "عالیه  من راضى بودم",
    "سلام  با این تخفیفی که داده و این قیمت به نظرم خیلی خوبه و می ارزه  ممنون از دیجی کالا  خیلی کرم پودر سبکو خوبیه ولی زود تموم میشه  Flower perfection بورژوا خیلی پوشش دهیش بهتره",
    "با توجه به مزایای این نوع پنکه خصوصا ثبات و کم صدا بودنش تو دور تندو بازدهی بالا  و ایرانی بودن محصول   برای کسانی که با پنکه با پایه بلند نیازی ندارند پیشنهاد میکنم",
    "من این لپ تاپ رو چند ماه پیش خریدم و الان خیلی ناراحتم هم صداش کمه هم شارژش زود تموم میشه  دی وی دی اکسترنالشم بعد از دو بار استفاده دیگه درش بسته نشد وقتی هم درش و خودم نگه دارم هم درست کار نمیکنه  پیشنهاد نمیکنم  قیمتش هم تقریبا زیاد بود انتظار لپ تاپ خیلی خوبی رو داشتم ولی",
    "سلام دوستان حدود 2 سالی هست که این گوشی رو دارم و باید بگم فوق العادست فقط از اون اول مشکل باتری داشتم تا الان که دیگه عمر باتریش تموم شده و باید تعویض بشه اگه استفادتون از گوشی کمه حتما پیشنهاد میکنم این گوشی رو و اگه مثه من همیشه گوشی دستتون هست باید بگم که اصلا انتخاب مناسبی نیست چون همه نقاط قوت این گوشی با روشن بودنش قابل استفادست در غیر این صورت هیچ  و در آخر یادم رفت بگم که زیبایی خاصی هم داره که خیلی ها با آیفون اشتباه میگیرنش"
]

# Initialize a list to store results
results = []

# Process each review and save results
for review in reviews:
    sentiment = get_sentiment(review)
    time.sleep(10)
    extractive_phrase = get_extractive_self_explanation(review)
    time.sleep(10)
    counterfactual = get_counterfactual_review(review)
    time.sleep(10)
    counterfactual_sentiment = get_sentiment(counterfactual)
    time.sleep(10)
    
    #show the results
    print(f"Review: {review}")
    print(f"Sentiment: {sentiment}")
    print(f"Most Important Phrase: {extractive_phrase}")
    print(f"Counterfactual Review: {counterfactual}")
    print(f"Counterfactual Sentiment: {counterfactual_sentiment}")
    print("-" * 80)

    # Store the results in a dictionary
    result = {
        "Original Review": review,
        "Original Sentiment": sentiment,
        "Most Important Phrase": extractive_phrase,
        "Counterfactual Review": counterfactual,
        "Counterfactual Sentiment": counterfactual_sentiment
    }
    
    # Append the result to the list
    results.append(result)
    
    # Wait between requests to avoid rate limits
    #time.sleep(10)

# Convert the list of results into a DataFrame
df = pd.DataFrame(results)

# Save the results to a CSV file
df.to_csv("sentiment_analysis_results.csv", index=False)

# Plotting the comparison
fig, ax = plt.subplots(figsize=(10, 6))
df[['Original Sentiment', 'Counterfactual Sentiment']].value_counts().unstack().plot(kind='bar', ax=ax)
plt.title("Comparison of Original and Counterfactual Sentiments")
plt.xlabel("Sentiment")
plt.ylabel("Count")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

KeyboardInterrupt: 